In [ ]:
# %% 1. IMPORTS & KONFIGURATION
import sys
import json
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
from pathlib import Path

# Pfade definieren (Angepasst auf das neue Clean-File)
DATA_PATH = Path("../data/processed/istdata_trains_clean.parquet")
STATION_PATH = Path("../data/external/dienststellen.csv")
BORDER_PATH = Path("../data/external/switzerland.geojson")

# Plot-Einstellungen
MIN_OBS_MONTH = 1000  # Mindestanzahl Züge pro Station für Monatsplot
MIN_OBS_DAY = 50      # Mindestanzahl Züge pro Station für Tagesplot
VMAX = 2.0            # Farbskala Limit (Minuten) für Konsistenz

# %% 2. HILFSFUNKTIONEN (Clean Code)

def load_and_prep_data(data_path, station_path):
    """Lädt Zugdaten und Stationen, bereinigt sie und verknüpft sie."""
    
    # A) Zugdaten laden & Delta berechnen
    # HINWEIS: 'istdata_trains_clean' enthält bereits nur Züge, Filter entfernt
    df = pl.scan_parquet(data_path).collect()

    df = df.with_columns([
        # Delta: Abfahrt - Ankunft. 
        # Positiv = Mehr Verspätung aufgebaut (schlecht). 
        # Negativ = Verspätung aufgeholt (gut/Puffer).
        ((pl.col("delay_departure_s") - pl.col("delay_arrival_s")) / 60).alias("delta_delay_min"),
        pl.col("BPUIC").cast(pl.Utf8) # Casting für Join sicherstellen
    ]).filter(
        pl.col("delta_delay_min").is_between(-60, 60) # Extreme Ausreisser filtern
    )

    # B) Stationsdaten (Geo) laden
    # Parsing der Geoposition (z.B. "47.123, 8.456")
    stops = pl.read_csv(station_path, separator=";")
    stops = stops.with_columns([
        pl.col("operatingPointKilometerMasterNumber").cast(pl.Utf8).alias("BPUIC"),
        pl.col("Geoposition").str.split(",").list.get(0).str.strip_chars().cast(pl.Float64).alias("lat"),
        pl.col("Geoposition").str.split(",").list.get(1).str.strip_chars().cast(pl.Float64).alias("lon")
    ]).select(["BPUIC", "lat", "lon"]).drop_nulls()

    return df, stops

def plot_switzerland_border(ax, json_path):
    """Zeichnet die Schweizer Grenze aus GeoJSON (Robustere Version)."""
    if not json_path.exists():
        return # Skip if file missing

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    # Standard GeoJSON hat "features", manche custom files haben direkt geometry
    features = data.get("features", [data]) 
    
    for feature in features:
        geom = feature.get("geometry", {})
        coords_list = geom.get("coordinates", [])
        geom_type = geom.get("type", "")

        # Flache Liste von Polygonen erzeugen
        polygons = []
        if geom_type == "Polygon":
            polygons = coords_list # Liste von Ringen
        elif geom_type == "MultiPolygon":
            for poly in coords_list:
                polygons.extend(poly) # Liste von Ringen hinzufügen
        
        # Zeichnen
        for ring in polygons:
            try:
                xs, ys = zip(*ring)
                ax.plot(xs, ys, color="#333333", linewidth=0.8, alpha=0.6)
            except:
                pass

def create_map_plot(data, title, filename=None):
    """Generische Funktion zum Erstellen der Karte."""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Hintergrund (Grenze)
    plot_switzerland_border(ax, BORDER_PATH)

    # Daten extrahieren
    lons = data["lon"].to_numpy()
    lats = data["lat"].to_numpy()
    vals = data["mean_delta"].to_numpy()
    sizes = data["n_obs"].to_numpy()

    # Scatter Plot
    # Blau (negativ) = Puffer/Aufholen, Rot (positiv) = Verspätungsaufbau
    sc = ax.scatter(
        lons, lats, c=vals, cmap="bwr",
        vmin=-VMAX, vmax=VMAX, 
        s=np.sqrt(sizes) * 1.5, # Punktgröße skalieren
        alpha=0.8, edgecolors="black", linewidth=0.1
    )
    
    # Layout
    cbar = plt.colorbar(sc, ax=ax, fraction=0.03, pad=0.04)
    cbar.set_label("Δ Verspätung (Min)\nblau = holt auf | rot = verliert Zeit")
    
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    
    # Zoom auf die Schweiz (grob)
    ax.set_xlim(5.9, 10.6)
    ax.set_ylim(45.8, 47.9)
    
    # Aspektverhältnis korrigieren (Mercator-ähnlich für CH)
    ax.set_aspect(1.5) 
    
    plt.tight_layout()
    plt.show()

# %% 3. DATEN LADEN
print(f"Lade Daten von {DATA_PATH}...")
df_trains, df_stops = load_and_prep_data(DATA_PATH, STATION_PATH)
print(f"Daten geladen: {df_trains.height} Fahrten, {df_stops.height} Stationen.")

# %% 4. PLOT: GESAMTER MONAT
print("Erstelle Monats-Plot...")

# Aggregation pro Station
agg_month = (
    df_trains
    .group_by("BPUIC")
    .agg([
        pl.col("delta_delay_min").mean().alias("mean_delta"),
        pl.col("delta_delay_min").count().alias("n_obs")
    ])
    .filter(pl.col("n_obs") >= MIN_OBS_MONTH) # Nur relevante Stationen
    .join(df_stops, on="BPUIC", how="inner")
)

create_map_plot(agg_month, "Netz-Performance: Wo entsteht Verspätung? (Gesamter Monat)")

# %% 5. PLOT: TAGES-SCHLEIFE (Optional)
print("Starte Tages-Analyse...")

# Wir plotten beispielhaft nur den ersten Tag, um nicht 30 Plots zu spammen
# Entfernen Sie [:1] um alle Tage zu plotten.
days = df_trains["BETRIEBSTAG"].unique().sort()[:1] 

for day in days:
    print(f"Plotting {day}...")
    
    agg_day = (
        df_trains
        .filter(pl.col("BETRIEBSTAG") == day)
        .group_by("BPUIC")
        .agg([
            pl.col("delta_delay_min").mean().alias("mean_delta"),
            pl.col("delta_delay_min").count().alias("n_obs")
        ])
        .filter(pl.col("n_obs") >= MIN_OBS_DAY)
        .join(df_stops, on="BPUIC", how="inner")
    )
    
    if agg_day.height > 0:
        create_map_plot(agg_day, f"Performance am {day}")
    else:
        print(f"Keine ausreichenden Daten für {day}")

Lade Daten von ../data/processed/istdata_trains_clean.parquet...


FileNotFoundError: No such file or directory (os error 2): ../data/processed/istdata_trains_clean.parquet

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'
